Install pyterrier it using pip (`pip install python-terrier`).

### Step 1: Initialize PyTerrier

First, you need to initialize PyTerrier. This is typically done once per notebook or script.

```python
import pyterrier as pt
if not pt.started():
    pt.init()
```

### Step 2: Dataset Loading

Load your dataset. You can use one of the datasets available in PyTerrier, or load your own dataset.

```python
# Example of loading a dataset from PyTerrier
dataset = pt.get_dataset('irds:your-dataset-name')
# For custom datasets, you'd typically load them into a dataframe or similar structure
```

### Step 3: Preprocessing and Query Rewriting

Implement the NLP-based query understanding and rewriting. You might use existing NLP libraries such as spaCy or Hugging Face's Transformers for this purpose.

```python
from your_query_rewriting_module import rewrite_query

# This is a placeholder function. You will implement your query understanding and rewriting logic here.
def preprocess_query(query):
    # Use NLP techniques to understand and rewrite the query
    rewritten_query = rewrite_query(query)
    return rewritten_query
```

### Step 4: Developing the IR System

Develop your prototype IR system. This involves setting up the indexing and retrieval process.

```python
# Indexing
indexer = pt.DFIndexer("./index_path", overwrite=True)
index_ref = indexer.index(dataset.get_corpus_iter())

# Retrieval
# You can use one of PyTerrier's built-in retrieval models or develop your own.
retriever = pt.BatchRetrieve(index_ref, wmodel="BM25")
```

### Step 5: Experimentation

Run experiments using your IR system. This includes query processing, retrieval, and evaluation against your benchmarks.

```python
from pyterrier.measures import RR, nDCG

# Example query processing and retrieval
def run_query_and_evaluate(query):
    rewritten_query = preprocess_query(query)
    result = retriever.transform(rewritten_query)
    # Evaluation - assuming you have a test set with relevance judgments
    eval_result = pt.Utils.evaluate(result, qrels, measures=[RR, nDCG])
    return eval_result

# Example usage
query = "Your test query"
evaluation_metrics = run_query_and_evaluate(query)
print(evaluation_metrics)
```

### Step 6: Analysis

Analyze the results to evaluate the effectiveness of your advanced query understanding and rewriting techniques.